In [1]:
import nltk
import parse_corpus
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import numpy as np

import lda

# Split into sections

In [2]:
filename = "preprocessed_shakespeare.txt"
castfile = "CleanedCast.txt"

sections = parse_corpus.process(filename)
castdict = {}
for line in open(castfile):
    words = nltk.word_tokenize(line.strip())
    for word in words:
        castdict[word] = True

In [3]:
# with open("/Users/markmartinez/Desktop/shakespeare.txt","r") as F:
#     allText = F.read().lower()

# Remove stopwords, tokenize, stem (takes about 30 seconds)

In [15]:
stops = stopwords.words("english")
stopdict = dict((s,None) for s in stops) # Sets are really terrible in Python
# print [s for s in stopdict.iterkeys()]

ps = PorterStemmer()

clean_sections = []

# NOTE(tfs): I am using tokenization now. Mainly it really cleans up punctuation and handles contractions well
for section in sections:
#     secwords = section.split()
#     tokens = nltk.word_tokenize(" ".join(secwords))
    tokens = nltk.word_tokenize(section)
    nonstops = [w for w in tokens if not (w in stopdict or w in castdict)]
    stemmed = [ps.stem(t.lower()) for t in nonstops if t.isalnum()]
#     nonstops = [w for w in stemmed if not (w in stopdict or w in castdict)]
    clean_sections.append(" ".join(stemmed)) # Note this does not preserve structure,
                                              #      but all words are now present in the section string
print clean_sections[0][:500]

rousillon the count palac enter bertram countess of rousillon helena lafeu black countess in deliv i buri husband bertram and i go madam weep death anew i must attend majesti command i ward evermor subject lafeu you shall find king husband madam he gener time good must necess hold virtu whose worthi would stir want rather lack abund countess what hope majesti amend lafeu he hath abandon physician madam whose practic hath persecut hope find advantag process lose hope countess thi o sad passag ski


In [16]:
# stops = set(stopwords.words("english"))
# words = allText.split()
# meaningful_words = [w for w in words if not w in stops]
# cleanedUp =  " ".join( meaningful_words )

In [17]:
# cleanedUp[0:500]

In [18]:
#now split on 'scene'
# scenes = [x.strip() for x in cleanedUp.split("scene")]

In [19]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word")

data = vectorizer.fit_transform(clean_sections)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = data.toarray()

vocab = vectorizer.get_feature_names()

In [20]:
print sum(train_data_features[300])
print len(vocab)
print vocab[100:300]

516
13388
[u'10', u'aaron', u'abaissiez', u'abandon', u'abas', u'abash', u'abat', u'abbess', u'abbey', u'abbot', u'abbrevi', u'abe', u'abel', u'abergavenni', u'abet', u'abhor', u'abhorr', u'abhorson', u'abi', u'abid', u'abil', u'abject', u'abjectli', u'abjur', u'abl', u'abler', u'aboard', u'abod', u'abomin', u'abort', u'abound', u'about', u'abov', u'abr', u'abraham', u'abram', u'abreast', u'abridg', u'abroach', u'abroad', u'abrog', u'abrook', u'abrupt', u'abruptli', u'absenc', u'absent', u'absey', u'absolut', u'absolv', u'abstain', u'abstemi', u'abstin', u'abstract', u'absurd', u'absyrtu', u'abu', u'abund', u'abundantli', u'abus', u'abysm', u'academ', u'accent', u'accept', u'access', u'accessari', u'accid', u'accident', u'accit', u'acclam', u'accommod', u'accommodo', u'accompani', u'accomplic', u'accomplish', u'accompt', u'accord', u'accordeth', u'accordingli', u'accost', u'account', u'accoutr', u'accru', u'accu', u'accumul', u'accur', u'accurs', u'accurst', u'accus', u'accusativo', u'

In [21]:
# vocab = vectorizer.get_feature_names()
# print(len(vocab))

print type(data)

<class 'scipy.sparse.csr.csr_matrix'>


In [22]:
# # import numpy as np

# # Sum up the counts of each vocabulary word
# dist = np.sum(data, axis=0)

# # For each, print the vocabulary word and the number of times it 
# # appears in the training set
# for tag, count in zip(vocab, dist):
#     print(count, tag)

# Run LDA (takes a bit of time)

In [23]:
# Settings
num_topics = 20
num_iter = 500
n_top_words = 5

In [24]:
model = lda.LDA(n_topics=num_topics, n_iter=num_iter)
model.fit(data)

topic_word = model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 643
INFO:lda:vocab_size: 13388
INFO:lda:n_words: 420044
INFO:lda:n_topics: 20
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -4754734
INFO:lda:<10> log likelihood: -3814275
INFO:lda:<20> log likelihood: -3671482
INFO:lda:<30> log likelihood: -3612177
INFO:lda:<40> log likelihood: -3571192
INFO:lda:<50> log likelihood: -3541326
INFO:lda:<60> log likelihood: -3519550
INFO:lda:<70> log likelihood: -3497999
INFO:lda:<80> log likelihood: -3481025
INFO:lda:<90> log likelihood: -3466987
INFO:lda:<100> log likelihood: -3457343
INFO:lda:<110> log likelihood: -3446237
INFO:lda:<120> log likelihood: -3436638
INFO:lda:<130> log likelihood: -3429910
INFO:lda:<140> log likelihood: -3420782
INFO:lda:<150> log likelihood: -3413559
INFO:lda:<160> log likelihood: -3407148
INFO:lda:<170> log likelihood: -3400221
INFO:lda:<180> log likelihood: -3396379
INFO:lda:<190> log likelihood: -3392249
INFO:lda:<200> log likelihood: -3387713
INFO:lda:<210> log likelihood: -3384276
INFO:lda:<

Topic 0: rosalind helena love parol orlando
Topic 1: sir falstaff page ford master
Topic 2: valentin proteu berown love julia
Topic 3: the of great enter macbeth
Topic 4: duke clown angelo olivia isabella
Topic 5: timon petruchio servant tranio and
Topic 6: portia antonio bassanio shylock ring
Topic 7: and titu thi son rome
Topic 8: imogen prospero th posthumu cymbelin
Topic 9: king gloucest queen york richard
Topic 10: enter and talbot fight the
Topic 11: king and franc princ shall
Topic 12: second first coriolanu citizen meneniu
Topic 13: thou thi thee art hast
Topic 14: iago othello cassio desdemona gentleman
Topic 15: to and that the my
Topic 16: come good well shall would
Topic 17: caesar antoni brutu cleopatra cassiu
Topic 18: of troilu syracus dromio antipholu
Topic 19: and love thi that the
